In [2]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import os
import re
from matplotlib import colors

In [49]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True) #add flag force_remount = True if needed

Mounted at /content/drive


In [51]:
!cp -rf "/content/drive/MyDrive/LDA/News.zip" .
!cp -rf "/content/drive/MyDrive/LDA/Headline.zip" .
!unzip -q News.zip
!unzip -q Headline.zip

replace News_Category_Dataset_v3.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [52]:
#df = pd.read_json('News_Category_Dataset_v3.json', orient = 'records', lines = True)
#df[['headline', 'category', 'short_description']].head()
df = pd.read_csv('abcnews-date-text.csv')
df.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

#preprocessing
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet

stopwordset = set(stopwords.words())
# for named entity recognition (NER)
from nltk import ne_chunk

In [14]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
def clean_text(headline):
  le=WordNetLemmatizer()
  word_tokens=word_tokenize(headline)
  tokens=[le.lemmatize(w) for w in word_tokens if w not in stopwordset]
  cleaned_text=" ".join(tokens)
  return cleaned_text

In [60]:
df['head_clean']=df['headline_text'].apply(clean_text)
#df['desc_clean']=df['short_description'].apply(clean_text)
#df = df.drop(['authors', 'date'], axis = 1)

In [61]:
df.sample(10)

,publish_date,headline_text,head_clean
903862,20140919,gympie council to invest 75k in cbd to boost,gympie council invest 75k cbd boost
835277,20131119,milk prices to fall,milk price fall
914619,20141107,danby another illiberal visitor slips under th...,danby illiberal visitor slip radar
1000860,20151218,meat prices butchers,meat price butcher
370357,20080316,fresh slush fund claims hit twu,fresh slush fund claim hit twu
637807,20110908,abjorensen when it comes to precedents and par...,abjorensen precedent parallel ; gilla
172436,20050628,hot enough ja,hot
74379,20040224,boost planned for offshore fisheries patrols,boost planned offshore fishery patrol
835256,20131119,liberal mps want regional development portfolio,liberal mp regional development portfolio
1176326,20190915,jew in the outback,jew outback


In [18]:
df.shape


(209527, 6)

In [62]:
#df.drop(['headline', 'short_description'], axis=1, inplace=True)
#df['clean_combined'] = df['head_clean'] + ' ' + df['desc_clean']
df.drop(['headline_text'], axis=1, inplace=True)
df.head()

,publish_date,head_clean
0,20030219,aba decides community broadcasting licence
1,20030219,act fire witness aware defamation
2,20030219,g call infrastructure protection summit
3,20030219,air nz staff aust strike pay rise
4,20030219,air nz strike affect australian traveller


In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
vect = TfidfVectorizer(stop_words=list(stopwordset), max_features=5000)

#vect_text=vect.fit_transform(df['clean_combined'])
vect_text=vect.fit_transform(df['head_clean'])
idf = vect.idf_

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aldiz', 'baiknya', 'baizik', 'berkali', 'bukatzeko', 'edota', 'eze', 'ezpabere', 'ezpada', 'ezperen', 'gainera', 'gainerontzean', 'guztiz', 'hainbestez', 'horra', 'kali', 'kurangnya', 'mata', 'olah', 'onların', 'ordea', 'osterantzean', 'printr', 'sekurang', 'setidak', 'tama', 'tidaknya', 'δι', 'арбаң', 'арсалаң', 'афташ', 'бай', 'бале', 'баски', 'батыр', 'баҳри', 'болои', 'бүгжең', 'бұтыр', 'валекин', 'вақте', 'вой', 'вуҷуди', 'гар', 'гарчанде', 'далаң', 'даме', 'ербелең', 'жалт', 'жұлт', 'карда', 'кошки', 'куя', 'күңгір', 'кӣ', 'магар', 'майлаш', 'митың', 'модоме', 'нияти', 'онан', 'оре', 'паһ', 'рӯи', 'салаң', 'сар', 'сұлаң', 'сұрт', 'тарбаң', 'тразе', 'ту', 'тыржың', 'тұрс', 'хом', 'хуб', 'чаро', 'чи', 'чун', 'чунон', 'шарте', 'шұңқ', 'ыржың', 'қадар', 'қайқаң', 'қалт', 'қаңғыр', 'қаңқ', 

In [64]:
dd=dict(zip(vect.get_feature_names_out(), idf))
l=sorted(dd, key=(dd).get)
# print(l)
print(l[0],l[-1])
print(dd['trump'])
print(dd['beer'])  # police is most common and forecast is least common among the news headlines.

police underworld
6.358598448961041
8.643750587985473


In [65]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=42,max_iter=10) 
lda_top=lda_model.fit_transform(vect_text)
print(lda_top.shape)  # (no_of_doc,no_of_topics)
print(lda_top)

(1244184, 10)
[[0.0415823  0.0415823  0.0415823  ... 0.0415823  0.0415823  0.62575927]
 [0.03140408 0.03140408 0.03140408 ... 0.03140408 0.62191844 0.03140408]
 [0.03362727 0.03362474 0.03362474 ... 0.03362474 0.03363289 0.21721026]
 ...
 [0.03363973 0.03363973 0.03363973 ... 0.16718878 0.03363973 0.35369275]
 [0.24114387 0.03373106 0.03373106 ... 0.03373106 0.03373106 0.03373106]
 [0.29551169 0.03107981 0.15337297 ... 0.03107981 0.03107981 0.16075523]]


In [71]:
print("Document 2: ")
for i,topic in enumerate(lda_top[2]):
  print("Topic ",i,": ",topic*100,"%")

Document 2: 
Topic  0 :  3.362726973483273 %
Topic  1 :  3.3624744107486024 %
Topic  2 :  3.362474410733749 %
Topic  3 :  3.3624744107327054 %
Topic  4 :  3.3624744107382685 %
Topic  5 :  51.37811196804476 %
Topic  6 :  3.362474410752134 %
Topic  7 :  3.362474410723092 %
Topic  8 :  3.3632885933390413 %
Topic  9 :  21.72102600070437 %


In [70]:
vocab = vect.get_feature_names_out()

for i, comp in enumerate(lda_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:20]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
woman covid world court 19 change country house charge life cup australia labor north funding talk afl murder guilty drum 

Topic 1: 
australian queensland home show worker former student green rate abuse png river bank vote star support town morrison pacific begin 

Topic 2: 
case child adelaide trial donald dead make sex federal protest victorian assault review arrested darwin season port drought andrew perth 

Topic 3: 
coronavirus trump coast found tasmania test final accused gold return tasmanian bushfire search leader inquiry court fight body appeal commission 

Topic 4: 
police interview death hit record time west ban driver find group royal tax station run officer fatal trade aussie art 

Topic 5: 
australia fire day election crash melbourne school abc nsw brisbane rise road power back flood game warning medium alleged scott 

Topic 6: 
plan china victoria national open farmer killed work industry farm fall park jailed council climate local regional young vaccine hill

<1x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 13 stored elements in Compressed Sparse Row format>